#Black Hole
###This game is described using a 10 X 10 grid :
>   _ _ _ _ _ _ _ _ _ H<br>   _ _ _ _ _ _ _ H _ _<br>   _ _ _ _ _ H _ _ _ _<br>   _ _ _ H _ _ _ _ _ _<br>   _ H _ _ _ _ _ _ _ _<br>   _ _ _ _ _ _ _ _ _ H <br>   _ _ _ _ _ _ _ H _ _<br>   _ _ _ _ _ H _ _ _ _ <br>   _ _ _ H _ _ _ _ _ _ <br>   G H _ _ _ _ _ _ _ _
<br><br> _ : Safe path
<br> H : Black Hole, avoid falling
<br> G: Goal, target to reach
###These black holes will be continuously moving toward left.
###At your each step they will move 1 step towards left.
###Your goal is to reach G and receive reward 1.
###You fall in black hole only if your position and black hole's position coincide.
###The episode ends when you reach G or fall in H.
###If you reach goal and black hole also reaches G, you loose.
##Reward : 
###&emsp;1 if you reach G<br>
###&emsp;-1 if you fell in black hole <br>
###&emsp;0 otherwise.


#**DO NOT EDIT THIS CODE**

In [ ]:
import random
import sys
import copy
class BlackHole:
  class action:
    def __init__(self):
      self.total_actions = 5
      self.dtype = type(self.total_actions)
      self.__out=sys.stdout
    
    def random_action(self):
      act = random.randint(1,5)
      return act

    def show_actions(self):
      actions= "1->Up, 2->Right, 3->Down, 4->Left 5->Stay"
      self.__out.write(actions)
    
  class observation:
    def __init__(self):
      self.total_observations = 1000
      self.dtype = type(self.total_observations)
      self.__lst =[x for x in range(1,1000) if x%100 is not 0]

    def random(self):
      obs = random.sample(self.__lst,1)[0]
      return obs
  def __init__(self):
    self.observation_space = self.observation()
    self.__map=['_________H',
                '_______H__',
                '_____H____',
                '___H______',
                '_H________',
                '_________H',
                '_______H__',
                '_____H____',
                '___H______',
                'GH________']
    self.action_space = self.action()
    self.__x = None
    self.__y = None
    self.__state = None
    self.__out = sys.stdout
    self.__action = None
    self.__action_dict = {1:'Up',2:'Right',3:'Down',4:'Left',5:'Stay'}
    self.__done = False
    self.__h = None

  def reset(self):
    self.__y = 0
    self.__h = 9
    self.__adjust_h()
    self.__x = random.randint(0,4)
    self.current_state()
    self.__action = None
    self.__done = False
    return self.__state

  def __adjust_h(self):
    for i in  range(len(self.__map)):
      if i==9 and self.__map[i].count('_')==9:
        self.__map[9] = self.__map[9][:9]
      self.__map[i] = ''.join(i for i in self.__map[i] if i is not 'H')
      index = self.__h-2*i 
      index  = index + (index<0 and index>-10)*10 + (index<-10)*20 
      self.__map[i] = self.__map[i][:index] + 'H' + self.__map[i][index:]
    if self.__h == 8:
      self.__map[9] = ''.join(i for i in self.__map[9] if i is not 'H')+'_'
  
  def current_state(self):
    if self.__y is not None:
     self.__state = (9-self.__h)*100+self.__y*10+self.__x+1
    return self.__state

  def take_step(self,action):
    if self.__done == False :
      reward = 0.0
      if action == 1:
        if self.__y-1>=0:
          self.__y-=1
        self.__action = action
      elif action == 3:
        if self.__y+1<=9:
          self.__y+=1
        self.__action = action
      elif action == 2:
        if self.__x+1<=9:
          self.__x+=1
        self.__action = action
      elif action == 4:
        if self.__x-1>=0:
          self.__x-=1
        self.__action = action
      elif action == 5:
        self.__action = action
      else:
        self.__out.write("Enter a valid action.")
        return
      self.__h  = self.__h -1 + (self.__h-1<0)*10
      self.__adjust_h()
      self.current_state()
      if self.__map[self.__y][self.__x]=='G' :
        reward=1.0
        self.__done= True
        if self.__h ==8:
          reward = -1.0
          self.__done = True
      if self.__map[self.__y][self.__x]=='H':
        reward = -1.0
        self.__done = True
      return self.__state,reward,self.__done
    else :
      self.__out.write("\n\033[38;5;11mWARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True\033[0;0m")

  def show(self):
    if self.__state is not None:
      map = copy.deepcopy(self.__map)
      val = map[self.__y][self.__x]
      map[self.__y] = map[self.__y][:self.__x] + 'P' +map[self.__y][self.__x+1:]
      map = self.__add_colour_h(map)
      map[-1]=map[-1].replace('G',"\033[38;5;12mG\033[0;0m")
      map[self.__y] = map[self.__y].replace('P',f'\033[48;5;9m{val}\033[0;0m')
      if self.__action is not None:
        self.__out.write('\n'+self.__action_dict[self.__action])
      self.__out.write("\n+----------+")
      for i in map:
        self.__out.write('\n|'+i+'|')
      self.__out.write("\n+----------+")
      if val =='H':
        self.__out.write("\nTRY AGAIN.......You fell in Black Hole!!!")
      if val =='G':
        if self.__h is not 8 : 
          self.__out.write("\nGG!!")
        else :
          self.__out.write("\nYou reached but fell in Black Hole")
      self.__out.write("\n")
    else :
      self.__out.write('NONE')

  def __add_colour_h(self,map):
    for i in range(len(map)):
      map[i]=map[i].replace('H','\033[48;5;16mH\033[0;0m')
    if self.__h == 8:
      map[9] = map[9].replace('G','\033[48;5;16mG\033[0;0m')
    return map

  def set_state(self,state):
    if state>1000 or state<1:
      self.__out.write("Enter a valid state.")
      return
    self.__state = state
    self.__h = 9 - (state-1)//100
    self.__y = ((state-1)%100)//10
    self.__x = ((state-1)%100)%10
    self.__adjust_h()
    if self.__map[self.__y][self.__x]=='_':
      self.__done = False
    else: 
      self.__done = True
    self.__action = None

#Environment methods and attributes

In [ ]:
env = BlackHole() #Creating object of BlackHole class

In [ ]:
print(env.observation_space.total_observations) #Total observations in observation space
print(env.observation_space.random()) # Random observation from observation space

1000
697


In [ ]:
print(env.action_space.total_actions) #Total actions in action space
print(env.action_space.random_action()) #Returns random action from action space
env.action_space.show_actions() #Prints details about actions in action space

5
2
1->Up, 2->Right, 3->Down, 4->Left 5->Stay

In [ ]:
print(env.current_state()) #No state is initailized

None


In [ ]:
env.reset() #initilizes game and you are spawned at one of first five blocks
env.show() #prints observation
print(env.current_state())


+----------+
|_________H|
|_______H__|
|_____H____|
|___H______|
|_H________|
|_________H|
|_______H__|
|_____H____|
|___H______|
|GH________|
+----------+
3


In [ ]:
env.set_state(env.observation_space.random()) #state of environment is changed to state specified
env.show() 
print(env.current_state())


+----------+
|H_________|
|________H_|
|______H___|
|____H_____|
|__H_______|
|H_________|
|________H_|
|______H___|
|____H_____|
|G_H_______|
+----------+
933


###env.take_step( ) returns THREE values only, state, reward and done (episode completed or not) 

In [ ]:
from IPython.display import clear_output
from time import sleep
env.reset()
done = False
while True:
  env.show()
  clear_output(wait=True)
  sleep(1.0)
  if done: 
    break
  action = env.action_space.random_action()
  state,reward,done = env.take_step(action)


Down
+----------+
|____H_____|
|__H_______|
|H_________|
|________H_|
|______H___|
|____H_____|
|__H_______|
|H_________|
|________H_|
|G_____H___|
+----------+
TRY AGAIN.......You fell in Black Hole!!!


##**Task :**
###Now you are familiar with BlackHole environment. You have to implement q-learing on this custom environment. Remember you are not allowed to do any chnages in BlackHole class.